In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from math import log2

def logistic_map(x, r=3.99):
    return r * x * (1 - x)

def generate_logistic_map_image(image_size=28, initial_value=0.4, r=3.99):
    """
    Генерирует одно изображение size×size из логистического отображения.
    Значения в [0, 1].
    """
    iterations = image_size * image_size
    x = initial_value
    seq = []
    for _ in range(iterations):
        x = logistic_map(x, r)
        seq.append(x)
    img = np.array(seq).reshape((image_size, image_size))
    return img

def generate_logistic_map_images_dataset(num_images, image_size=28, r=3.99, fixed_initial=False):
    """
    Датасет из num_images карт логистического отображения.
    Если fixed_initial=False — каждая картинка со своим случайным x0.
    """
    dataset = []
    for _ in range(num_images):
        init_val = 0.4 if fixed_initial else np.random.rand()
        img = generate_logistic_map_image(image_size=image_size, initial_value=init_val, r=r)
        dataset.append(img)
    dataset = np.array(dataset)
    return dataset[..., np.newaxis].astype("float32")


def build_dense_autoencoder(image_size=(28, 28), latent_dim=64):
    """
    ЧИСТЫЙ baseline:
    - только Dense слои
    - только ReLU
    - НИКАКОЙ sparsity, L1, chaos, TopK и т.д.
    """
    h, w = image_size
    input_img = keras.Input(shape=(h, w, 1), name="input")
    x = layers.Flatten(name="flatten")(input_img)

    # Энкодер
    x = layers.Dense(256, activation="relu", name="enc_dense_1")(x)
    x = layers.Dense(128, activation="relu", name="enc_dense_2")(x)
    latent = layers.Dense(latent_dim, activation="relu", name="latent")(x)

    encoder = keras.Model(input_img, latent, name="dense_encoder")

    # Декодер
    x = layers.Dense(128, activation="relu", name="dec_dense_1")(latent)
    x = layers.Dense(256, activation="relu", name="dec_dense_2")(x)
    x = layers.Dense(h * w, activation="sigmoid", name="dec_out")(x)
    decoded = layers.Reshape((h, w, 1), name="output")(x)

    autoencoder = keras.Model(input_img, decoded, name="dense_autoencoder")
    autoencoder.compile(optimizer=keras.optimizers.Adam(1e-3), loss="mse")

    return autoencoder, encoder


def analyze_latent_statistics(encoder, images, zero_threshold=1e-6, verbose=True):
    """
    Считает:
      - дисперсию по измерениям
      - среднюю дисперсию
      - долю “мёртвых” нейронов (всегда ~0)
    """
    latents = encoder.predict(images, verbose=0)  # (N, D)
    latents = np.asarray(latents)
    variance_per_dim = np.var(latents, axis=0)
    mean_variance = float(np.mean(variance_per_dim))

    # "Мёртвый" нейрон: |h_i| < eps почти для всех сэмплов
    dead_mask = np.all(np.abs(latents) < zero_threshold, axis=0)
    dead_neurons = int(np.sum(dead_mask))
    total_neurons = latents.shape[1]

    if verbose:
        print("\n===== Dense AE Latent Statistics =====")
        print(f"Latent shape: {latents.shape}")
        print(f"Mean variance per dimension: {mean_variance:.6e}")
        print(f"Dead neurons: {dead_neurons}/{total_neurons} ({dead_neurons/total_neurons:.1%})")
        print("First 10 variances:", variance_per_dim[:10])

    stats = {
        "variance_per_dim": variance_per_dim,
        "mean_variance": mean_variance,
        "dead_neurons": dead_neurons,
        "total_neurons": total_neurons,
    }
    return stats


def test_latent_chaos_behavior_dense(encoder,
                                     num_steps=10,
                                     image_size=28,
                                     r=3.99,
                                     init=0.4,
                                     delta=1e-5):
    """
    Два близких начальных значения x0 и x0+δ, на каждом шаге генерируем картинку,
    пропускаем через encoder и считаем ||z1_t - z2_t||.

    Это прямой аналог твоего теста для стандартного AE, но для Dense ReLU baseline.
    """
    chain1 = []
    chain2 = []
    x1 = init
    x2 = init + delta

    for _ in range(num_steps):
        img1 = generate_logistic_map_image(image_size=image_size, initial_value=x1, r=r)
        img2 = generate_logistic_map_image(image_size=image_size, initial_value=x2, r=r)
        chain1.append(img1)
        chain2.append(img2)
        x1 = logistic_map(x1, r=r)
        x2 = logistic_map(x2, r=r)

    chain1 = np.array(chain1)[..., np.newaxis].astype("float32")
    chain2 = np.array(chain2)[..., np.newaxis].astype("float32")

    latent1 = encoder.predict(chain1, verbose=0)
    latent2 = encoder.predict(chain2, verbose=0)

    distances = [float(np.linalg.norm(latent1[i] - latent2[i])) for i in range(num_steps)]

    print("\n===== Dense AE Latent Chaos Divergence =====")
    for i, d in enumerate(distances):
        print(f"step {i}: distance = {d:.6f}")
    if distances[0] > 0:
        ratio = distances[-1] / distances[0]
        print(f"Final/initial ratio: {ratio:.3f}")
    else:
        print("Initial distance is 0 → ratio undefined.")

    return distances


def run_dense_ae_baseline_experiment(
    num_train=5000,
    num_val=1000,
    image_size=28,
    latent_dim=64,
    epochs=10,
    batch_size=64,
    r=3.99,
):
    """
    Полный эксперимент baseline Dense AE:
      1) генерация train/val набора логистических карт
      2) обучение Dense AE (ReLU)
      3) оценка reconstruction MSE
      4) анализ дисперсии латента
      5) тест хаотического расхождения
    """
    print("\n=== [1] Генерация данных для baseline Dense AE ===")
    train_images = generate_logistic_map_images_dataset(num_train, image_size=image_size, r=r, fixed_initial=False)
    val_images = generate_logistic_map_images_dataset(num_val, image_size=image_size, r=r, fixed_initial=False)

    print("\n=== [2] Построение Dense Autoencoder (ReLU) ===")
    autoencoder, encoder = build_dense_autoencoder(image_size=(image_size, image_size), latent_dim=latent_dim)
    autoencoder.summary()

    print("\n=== [3] Обучение Dense AE ===")
    history = autoencoder.fit(
        train_images,
        train_images,
        validation_data=(val_images, val_images),
        epochs=epochs,
        batch_size=batch_size,
        verbose=1,
    )

    final_loss = history.history["loss"][-1]
    final_val_loss = history.history["val_loss"][-1]
    print(f"\nFinal train loss (MSE):     {final_loss:.6f}")
    print(f"Final validation loss (MSE): {final_val_loss:.6f}")

    print("\n=== [4] Анализ латентного пространства ===")
    stats = analyze_latent_statistics(encoder, val_images, verbose=True)

    print("\n=== [5] Тест хаотического расхождения ===")
    distances = test_latent_chaos_behavior_dense(encoder, num_steps=10, image_size=image_size, r=r)

    results = {
        "autoencoder": autoencoder,
        "encoder": encoder,
        "history": history.history,
        "latent_stats": stats,
        "chaos_distances": distances,
    }
    return results


if __name__ == "__main__":
    results = run_dense_ae_baseline_experiment(
        num_train=5000,
        num_val=1000,
        image_size=28,
        latent_dim=64,
        epochs=10,
        batch_size=64,
        r=3.99,
    )

    # Быстрый вывод ключевых чисел для включения в таблицу статьи
    print("\n=== SUMMARY FOR PAPER TABLE ===")
    print(f"Mean latent variance: {results['latent_stats']['mean_variance']:.6e}")
    print(f"Dead neurons: {results['latent_stats']['dead_neurons']}/{results['latent_stats']['total_neurons']}")
    print(f"Chaos distances: {results['chaos_distances']}")



=== [1] Генерация данных для baseline Dense AE ===

=== [2] Построение Dense Autoencoder (ReLU) ===
Model: "dense_autoencoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (InputLayer)          [(None, 28, 28, 1)]       0         
                                                                 
 flatten (Flatten)           (None, 784)               0         
                                                                 
 enc_dense_1 (Dense)         (None, 256)               200960    
                                                                 
 enc_dense_2 (Dense)         (None, 128)               32896     
                                                                 
 latent (Dense)              (None, 64)                8256      
                                                                 
 dec_dense_1 (Dense)         (None, 128)               8320      
              